In [3]:
import requests

##### Using 'API-KEY' to access the GFN API


In [1]:
api_key = 'G4h5oGDCacaNmpaPaU0sq9k4t58E58PNM0bdCb4Uu1785k3EqMq' # API key (required, can be obtained using https://data.footprintnetwork.org/#/api)

#url = 'https://api.footprintnetwork.org/v1/years'
request_headers = {"HTTP_ACCEPT": "application/json"}
username = 'footprint' # dummy username

def make_gfn_request(url):
    response = requests.get(url,headers=request_headers, auth=(username, api_key))
    if response.status_code == 200:
        return response.json()
                   

##### Extracting data from the API and pre-processing it before storing in a 'csv' file.

In [17]:
import pandas as pd

# to scale down the high-precision floating point numbers
def round_format(df):
    for col in df.columns:
        if isinstance(df[col].iloc[0], float):
            df[col] = df[col].apply(lambda x:round(x, 3))


#['value', 'year', 'score', 'shortName', 'countryCode', 'countryName', 'isoa2', 'record', 'cropLand', 'grazingLand', 'forestLand', 'fishingGround', 'builtupLand', 'carbon']
column_names = ['shortName', 'isoa2'] # omitting these column names
main_df = pd.DataFrame()
for year in range(2022, 1990, -5):
    url = f'https://api.footprintnetwork.org/v1/data/all/{year}'
    data_json = make_gfn_request(url)
    df = pd.json_normalize(data_json)
    df = df.drop(column_names, axis=1)
    round_format(df)
    main_df = pd.concat([main_df, df], ignore_index=True)
    
print(main_df.tail())
main_df.to_csv('main_df.csv', index=False) # temporarily store the combined data frame for network construction

       value  year score  countryCode                      countryName  \
27491  2.732  1992               2002                           Europe   
27492  1.277  1992               2003  Latin America and the Caribbean   
27493  4.691  1992               2004                    North America   
27494  1.228  1992               2005                          Oceania   
27495  1.206  1992    3A         5001                            World   

       record  cropLand  grazingLand  forestLand  fishingGround  builtupLand  \
27491  Earths     0.555        0.140       0.295          0.126        0.062   
27492  Earths     0.235        0.401       0.193          0.059        0.050   
27493  Earths     0.728        0.202       0.641          0.073        0.022   
27494  Earths     0.246        0.117       0.098          0.365        0.154   
27495  Earths     0.247        0.095       0.162          0.051        0.023   

       carbon  
27491   1.554  
27492   0.340  
27493   3.026  
27494   0.